### Plotly Figure Friday - 2024 week 47 - UFOs

In [194]:
'''
This script produces a scatter map of UFO sighting in North America, with
informative hover that includes all of the comment fields.

North America is represents over 84.8% of the data, and is expected to reach 95%
after fixing country values of null that match US  or Canadien states. 

This show the values counts and percentages by country.
┌─────────┬───────┬──────┐
│ country ┆ count ┆ PCT  │
│ ---     ┆ ---   ┆ ---  │
│ str     ┆ u32   ┆ f64  │
╞═════════╪═══════╪══════╡
│ us      ┆ 65114 ┆ 81.1 │
│ null    ┆ 9670  ┆ 12.0 │
│ ca      ┆ 3000  ┆ 3.7  │
│ gb      ┆ 1905  ┆ 2.4  │
│ au      ┆ 538   ┆ 0.7  │
│ de      ┆ 105   ┆ 0.1  │
└─────────┴───────┴──────┘
'''

import plotly.express as px
import polars as pl
import us
import pycountry

# constants 
csv_source = 'scrubbed.csv'

# functions
def get_state_list(df, country_abbr, country_col, state_col):
    state_abbr_list = (
        df
        .filter(pl.col(country_col) == country_abbr)
        .select(pl.col(state_col, country_col))
        .filter(pl.col(state_col).is_not_null())
        .unique(state_col)
        .select(pl.col(state_col))
        .to_series().to_list()
    )
    return(state_abbr_list)

#------------------------------------------------------------------------------#
#     with us library, make dataframe of state abbreviations and names         #
#------------------------------------------------------------------------------#
df_us_state_names = (
    pl.concat(  # use concat to add a row for Washington DC
        [
            pl.DataFrame(us.states.mapping('abbr', 'name'))
            .transpose(include_header=True)
            .rename({'column': 'STATE_ABBR', 'column_0': 'US_STATE'})
            ,
            pl.DataFrame(  # add 1-row data frame common abbr for Wash DC.
                {
                    'STATE_ABBR' : 'DC',
                    'US_STATE' : 'Washington DC'
                }
            )
        ]
    )
   
)

df_canadien_state_name = (

)


#------------------------------------------------------------------------------#
#     Read Data, minor cleanup                                                 #
#------------------------------------------------------------------------------#

df = (
    pl.scan_csv(
        csv_source,
        ignore_errors = True,
        try_parse_dates=True  # this converts first date from string to datetime
    )
    .filter(
        pl.col('country').is_in(['us', 'ca']) 
        |
        pl.col('country').is_null()
    )
    .with_columns(
        # in the csv if formated as Month/Day/Year. After cast of 
        # the string to a date, the format is shown as Year-Month-Day
        pl.col('date posted').str.to_date('%m/%d/%Y'),
        pl.col('country').str.to_uppercase(),
        pl.col('state').str.to_uppercase(),
        pl.col('city').str.to_titlecase(),
        # # pl.col('datetime').str.to_datetime("%b/%d/%Y",strict=False)
        # # pl.col('datetime').str.to_datetime('%b/%d/%Y %H:%M') #,strict=False)
        # pl.col('datetime').str.to_date('%b/%d/%Y %H:%M') #,strict=False)
    )
    .rename({'state': 'STATE_ABBR', 'country': 'COUNTRY_ABBR'})
    .select(pl.all().exclude('duration (hours/min)'))
    # .with_columns(
    #     country = pl.when(pl.col('STATE_ABBR').is_in(us_state_repairs))
    #                 .then(pl.lit('us'))
    #                 .otherwise('country')
    # )
    # .with_columns(
    #     country = pl.when(pl.col('STATE_ABBR').is_in(canadian_state_repairs))
    #                 .then(pl.lit('ca'))
    #                 .otherwise('country')
    # )
    # .join(
    #      df_state_names,
    #      on='STATE_ABBR',
    #      how='left'
    # )
    .collect()
)
# df.sample(25)
# print(
#     df.filter(
#         pl.col('COUNTRY_ABBR').is_null() 
#         & 
#         pl.col('STATE_ABBR').is_not_null())
#     .unique('STATE_ABBR')
#     .select(pl.col('STATE_ABBR', 'COUNTRY_ABBR'))
#     # sorted(
#     #     list(
#     #     df.filter(
#     #         pl.col('COUNTRY_ABBR').is_null() 
#     #         & 
#     #         pl.col('STATE_ABBR').is_not_null())
#     #     .unique('STATE_ABBR')
#     #     .select(pl.col('STATE_ABBR', 'COUNTRY_ABBR'))
#     #     )
#     # )
#     .sample(20)
# )
# #df.sample(20)

#------------------------------------------------------------------------------#
#     make list of each countries state names, use later to fix null values  #
#------------------------------------------------------------------------------#
# us_state_abbrs = sorted(list(
#     df
#     .filter(pl.col('COUNTRY_ABBR') == 'US')
#     .filter(pl.col('STATE_ABBR').is_not_null())
#     .select(pl.col('STATE_ABBR'))
#     .unique('STATE_ABBR')
#     # .select(pl.col('STATE_ABBR'))
# ))
# print(f'{list(us_state_abbrs) = }')
# # df.head()

# #------------------------------------------------------------------------------#
# #     Fix country names with list of known state or provinces of each country  #
# #------------------------------------------------------------------------------#
us_state_abbrs =  get_state_list(df, 'US', 'COUNTRY_ABBR', 'STATE_ABBR')
print(f'{len(us_state_abbrs) = }')
print(f'{sorted(us_state_abbrs) = }')
canada_state_abbrs =  get_state_list(df, 'CA', 'COUNTRY_ABBR', 'STATE_ABBR')
print(f'{len(canada_state_abbrs) = }')
print(f'{sorted(canada_state_abbrs) = }')

print(f'{set.intersection(set(us_state_abbrs), set(canada_state_abbrs)) = }')
print(1/0)

# canada_state_abbrs = (
#     df
#     .filter(pl.col('COUNTRY_ABBR') == 'CA')
#     .select(pl.col('STATE_ABBR', 'COUNTRY_ABBR'))
#     .filter(pl.col('STATE_ABBR').is_null())
#     .unique('STATE_ABBR')
#     .select(pl.col('STATE_ABBR'))
#     .to_series().to_list()
# )
# print(f'{canada_state_abbrs = }')


# df.filter(
#     pl.col('STATE_ABBR').is_in(canada_state_abbrs)
#     &
#     pl.col('COUNTRY_ABBR').is_null()
# ).sample(25)
# df.head()

len(us_state_abbrs) = 52
sorted(us_state_abbrs) = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
len(canada_state_abbrs) = 15
sorted(canada_state_abbrs) = ['AB', 'BC', 'MB', 'NB', 'NF', 'NS', 'NT', 'ON', 'PE', 'PQ', 'QC', 'SA', 'SK', 'YK', 'YT']
set.intersection(set(us_state_abbrs), set(canada_state_abbrs)) = set()


ZeroDivisionError: division by zero

In [ ]:
#------------------------------------------------------------------------------#
#     Read Data, minor cleanup                                                 #
#------------------------------------------------------------------------------#
 
df_value_counts = (
    pl.scan_csv(
        csv_source,
        ignore_errors = True,
        try_parse_dates=True  # this converts first date from string to datetime
    )
    .select(pl.col('country'))
    .collect()
    .to_series()
    .value_counts()
    .sort('count', descending=True)
    .with_columns(PCT = (100* pl.col('count')/pl.col('count').sum()).round(1))
)#.value_counts('country')
print(df_value_counts)



shape: (6, 3)
┌─────────┬───────┬──────┐
│ country ┆ count ┆ PCT  │
│ ---     ┆ ---   ┆ ---  │
│ str     ┆ u32   ┆ f64  │
╞═════════╪═══════╪══════╡
│ us      ┆ 65114 ┆ 81.1 │
│ null    ┆ 9670  ┆ 12.0 │
│ ca      ┆ 3000  ┆ 3.7  │
│ gb      ┆ 1905  ┆ 2.4  │
│ au      ┆ 538   ┆ 0.7  │
│ de      ┆ 105   ┆ 0.1  │
└─────────┴───────┴──────┘


In [ ]:
#------------------------------------------------------------------------------#
#     Fix country names with list of known state or provinces of each country  #
#------------------------------------------------------------------------------#
us_state_abbrs = sorted(
    df
    .filter(pl.col('country') == 'us')
    .unique('STATE_ABBR')
    .select(pl.col('STATE_ABBR'))
    .to_series().to_list()
)
print(f'{us_state_abbrs = }')

ColumnNotFoundError: unable to find column "country"; valid columns: ["datetime", "city", "STATE_ABBR", "COUNTRY_ABBR", "shape", "duration (seconds)", "comments", "date posted", "latitude", "longitude"]

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'filter' <---
DF ["datetime", "city", "STATE_ABBR", "COUNTRY_ABBR"]; PROJECT */10 COLUMNS; SELECTION: None

In [ ]:
df = (
    df
    .with_columns(
        country = pl.when(pl.col('STATE_ABBR').is_in(us_state_abbrs))
                    .then(pl.lit('us'))
                    .otherwise('country')
    )
)
print(
    sorted(
        list(
        df.filter(
            pl.col('country').is_null() 
            & 
            pl.col('STATE_ABBR').is_not_null())
        .unique('STATE_ABBR')
        .select(pl.col('STATE_ABBR'))
        )
    )
)


[shape: (67,)
Series: 'STATE_ABBR' [str]
[
	"WI"
	"SD"
	"RI"
	"KY"
	"TX"
	…
	"AZ"
	"NT"
	"MN"
	"NH"
	"TN"
]]


In [ ]:
print(
    sorted(
        list(
        df.filter(
            pl.col('country').is_null() 
            & 
            pl.col('STATE_ABBR').is_not_null())
        .unique('STATE_ABBR')
        .select(pl.col('STATE_ABBR'))
        )
    )
)

[shape: (67,)
Series: 'STATE_ABBR' [str]
[
	"WV"
	"AB"
	"NH"
	"CO"
	"AZ"
	…
	"FL"
	"MI"
	"AR"
	"MO"
	"TX"
]]


In [ ]:
#------------------------------------------------------------------------------#
#     with us library, make dataframe of state abbreviations and names         #
#------------------------------------------------------------------------------#
df_state_names = (
    pl.concat(  # use concat to add a row for Washington DC
        [
            pl.DataFrame(us.states.mapping('abbr', 'name'))
            .transpose(include_header=True)
            .rename({'column': 'STATE_ABBR', 'column_0': 'US_STATE'})
            ,
            pl.DataFrame(  # add 1-row data frame common abbr for Wash DC.
                {
                    'STATE_ABBR' : 'DC',
                    'US_STATE' : 'Washington DC'
                }
            )
        ]
    )
   
  
)
df_state_names.sort('STATE_ABBR')

STATE_ABBR,US_STATE
str,str
"""AK""","""Alaska"""
"""AL""","""Alabama"""
"""AR""","""Arkansas"""
"""AS""","""American Samoa"""
"""AZ""","""Arizona"""
…,…
"""VT""","""Vermont"""
"""WA""","""Washington"""
"""WI""","""Wisconsin"""


In [ ]:
#------------------------------------------------------------------------------#
#     with us library, make dataframe of state abbreviations and names         #
#------------------------------------------------------------------------------#
df_state_names = (
    pl.concat(  # use concat to add a row for Washington DC
        [
            pl.DataFrame(us.states.mapping('abbr', 'name'))
            .transpose(include_header=True)
            .rename({'column': 'STATE_ABBR', 'column_0': 'US_STATE'})
            ,
            pl.DataFrame(  # add 1-row data frame common abbr for Wash DC.
                {
                    'STATE_ABBR' : 'DC',
                    'US_STATE' : 'Washington DC'
                }
            )
        ]
    )
   
  
)
df_state_names.sort('STATE_ABBR')

STATE_ABBR,US_STATE
str,str
"""AK""","""Alaska"""
"""AL""","""Alabama"""
"""AR""","""Arkansas"""
"""AS""","""American Samoa"""
"""AZ""","""Arizona"""
…,…
"""VT""","""Vermont"""
"""WA""","""Washington"""
"""WI""","""Wisconsin"""


In [ ]:
#------------------------------------------------------------------------------#
#     with pycounry library, make df to map country abbreviations to names     #
#------------------------------------------------------------------------------#
my_country_abbrs = list(set(df['country']))
print(my_country_abbrs)
print(1/0)
# country = pycountry.countries.get(alpha_3]2='US')
print(f"Name: {country.name}, Official Name: {country.official_name}, Numeric: {country.numeric}")
for country in pycountry.countries:
    print(f"Name: {country.name}, Alpha-2: {country.alpha_2}, Alpha-3: {country.alpha_3}")
# df_country_names = (
# )

# df_state_names = (
#     pl.concat(  # use concat to add a row for Washington DC
#         [
#             pl.DataFrame(us.states.mapping('abbr', 'name'))
#             .transpose(include_header=True)
#             .rename({'column': 'STATE_ABBR', 'column_0': 'US_STATE'})
#             ,
#             pl.DataFrame(  # add 1-row data frame common abbr for Wash DC.
#                 {
#                     'STATE_ABBR' : 'DC',
#                     'US_STATE' : 'Washington DC'
#                 }
#             )
#         ]
#     )
   
  
# )
# df_state_names.sort('STATE_ABBR')

['GB', 'DE', None, 'US', 'AU', 'CA']


ZeroDivisionError: division by zero

In [ ]:
#------------------------------------------------------------------------------#
#     with us library, make dataframe of state abbreviations and names         #
#------------------------------------------------------------------------------#
df_state_names = (
    pl.concat(  # use concat to add a row for Washington DC
        [
            pl.DataFrame(us.states.mapping('abbr', 'name'))
            .transpose(include_header=True)
            .rename({'column': 'STATE_ABBR', 'column_0': 'US_STATE'})
            ,
            pl.DataFrame(  # add 1-row data frame common abbr for Wash DC.
                {
                    'STATE_ABBR' : 'DC',
                    'US_STATE' : 'Washington DC'
                }
            )
        ]
    )
   
  
)
df_state_names.sort('STATE_ABBR')

STATE_ABBR,US_STATE
str,str
"""AK""","""Alaska"""
"""AL""","""Alabama"""
"""AR""","""Arkansas"""
"""AS""","""American Samoa"""
"""AZ""","""Arizona"""
…,…
"""VT""","""Vermont"""
"""WA""","""Washington"""
"""WI""","""Wisconsin"""


In [ ]:
df.select(pl.col('duration (hours/min)').value_counts())

ColumnNotFoundError: duration (hours/min)

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'select' <---
DF ["datetime", "city", "STATE_ABBR", "country"]; PROJECT */11 COLUMNS; SELECTION: None

In [ ]:
df

datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
str,str,str,str,str,i64,str,str,str,f64,f64
"""10/10/1949 20:30""","""San Marcos""","""TX""","""US""","""cylinder""",2700,"""45 minutes""","""This event took place in early…","""4/27/2004""",29.883056,-97.941111
"""10/10/1949 21:00""","""Lackland Afb""","""TX""",null,"""light""",7200,"""1-2 hrs""","""1949 Lackland AFB&#44 TX. Lig…","""12/16/2005""",29.38421,-98.581082
"""10/10/1955 17:00""","""Chester (Uk/England)""",null,"""GB""","""circle""",20,"""20 seconds""","""Green/Orange circular disc ove…","""1/21/2008""",53.2,-2.916667
"""10/10/1956 21:00""","""Edna""","""TX""","""US""","""circle""",20,"""1/2 hour""","""My older brother and twin sist…","""1/17/2004""",28.978333,-96.645833
"""10/10/1960 20:00""","""Kaneohe""","""HI""","""US""","""light""",900,"""15 minutes""","""AS a Marine 1st Lt. flying an …","""1/22/2004""",21.418056,-157.803611
…,…,…,…,…,…,…,…,…,…,…
"""9/9/2013 21:15""","""Nashville""","""TN""","""US""","""light""",600,"""10 minutes""","""Round from the distance/slowly…","""9/30/2013""",36.165833,-86.784444
"""9/9/2013 22:00""","""Boise""","""ID""","""US""","""circle""",1200,"""20 minutes""","""Boise&#44 ID&#44 spherical&#44…","""9/30/2013""",43.613611,-116.2025
"""9/9/2013 22:00""","""Napa""","""CA""","""US""","""other""",1200,"""hour""","""Napa UFO&#44""","""9/30/2013""",38.297222,-122.284444


In [ ]:
print(1/0)

ZeroDivisionError: division by zero

In [ ]:
import plotly.express as px
import polars as pl

# constants
SOURCE_LOCAL = True # if True, data from csv, if False data from get git-repo
csv_local = 'week_46_data.csv'

csv_git_source = 'https://raw.githubusercontent.com/plotly/Figure-Friday/refs/'
csv_git_source += 'heads/main/2024/week-46/PDO_wine_data_IT_FR.csv'

wine_colors = {
    'Rosé'  : '#E3AFA7',  # Rose colors vary, extracted this from a picture
    'Red'   : '#9B2242',  # matches color named winered
    'White' : '#E7DF99',  # white wine is not white, this is deep charconnay
    }

#------------------------------------------------------------------------------#
#     initialize dataframe df_source from local file or git repo
#------------------------------------------------------------------------------#
if SOURCE_LOCAL:   # read cleand-up data from local directory
    df = pl.read_csv(csv_local)       
else:             # read source data from git_repo, and clean-up
    df = (
        pl.read_csv(csv_git_source)
        .with_columns(
            pl.col('Max_yield_hl')
                .cast(pl.UInt16, strict=False),  # False changes na to null         
            pl.col('Country')
                .str.replace('FR', 'France')
                .str.replace('IT', 'Italy')
        )
        .drop_nulls(subset='Max_yield_hl')
        .select(pl.all().exclude('PDOid', 'Info'))
        .with_columns(
            COLOR_RGB = pl.when(pl.col('COLOR') == 'Rosé')
                           .then(pl.lit('#FF0080'))
        )
    )
    df.write_csv(csv_local)
    df.head()

fig = px.violin(
    df,
    x='Color',
    y='Max_yield_hl',
    facet_col='COUNTRY',
    title = (
        'Maximum permitted wine yield (hectoliters per hectare) in France and Italy'
        '<a href="https://en.wikipedia.org/wiki/Yield_(wine)" ' + 
        'style="color:yellow;"> Wikipedia LINK</a>'
    ),
    color='Color', 
    color_discrete_map=wine_colors,
    template='plotly_dark',
)

fig.update_layout(
    font=dict(size=16), 
    showlegend=False,
    # xaxis_title=dict(text='Date', font=dict(size=16, color='#FFFFFF')),
    yaxis_title=dict(text=''),
)

# next line changes facet labels from COUNTRY=xyz, to just show xyz
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("COUNTRY=", "")))

# # this syntax is specific to the faceted plot
fig.update_xaxes(title='')

fig.show()
fig.write_html('Wines.html')